In [58]:
import yfinance as yf
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [59]:
# Barra covariance matrix

barra = pd.read_csv("grad_cov_matrix_excel.csv", index_col=0) # From excel

barra = barra.reset_index()

tickers = barra.columns.to_list()

barra

,AAPL,C,CMG,COKE,DRI,KLAC,MP,MTN,NFLX,GEF,RSG,_ETF29
0,0.066223,0.021693,0.019139,0.012855,0.016971,0.045822,0.050579,0.028011,0.024211,0.021417,0.011764,0.025473
1,0.021693,0.094066,0.018357,0.012294,0.028692,0.034565,0.066176,0.048799,0.025784,0.041188,0.015395,0.029396
2,0.019139,0.018357,0.069164,0.016344,0.031006,0.033433,0.038168,0.032852,0.025877,0.013959,0.013763,0.022240
3,0.012855,0.012294,0.016344,0.111396,0.015362,0.021428,0.024784,0.017683,0.015660,0.015072,0.011720,0.015285
4,0.016971,0.028692,0.031006,0.015362,0.054100,0.027188,0.045185,0.040054,0.019921,0.023303,0.014767,0.021559
5,0.045822,0.034565,0.033433,0.021428,0.027188,0.131522,0.080148,0.047370,0.039233,0.028484,0.016177,0.040736
6,0.050579,0.066176,0.038168,0.024784,0.045185,0.080148,0.332083,0.089390,0.052683,0.061946,0.014247,0.055600
7,0.028011,0.048799,0.032852,0.017683,0.040054,0.047370,0.089390,0.120935,0.032193,0.039318,0.016052,0.034572
8,0.024211,0.025784,0.025877,0.015660,0.019921,0.039233,0.052683,0.032193,0.136735,0.017499,0.010921,0.026648
9,0.021417,0.041188,0.013959,0.015072,0.023303,0.028484,0.061946,0.039318,0.017499,0.085402,0.014079,0.024153


In [60]:
# Get covariance matrix

cov_matrix = barra.to_numpy()

cov_matrix

array([[0.066223, 0.021693, 0.019139, 0.012855, 0.016971, 0.045822,
        0.050579, 0.028011, 0.024211, 0.021417, 0.011764, 0.025473],
       [0.021693, 0.094066, 0.018357, 0.012294, 0.028692, 0.034565,
        0.066176, 0.048799, 0.025784, 0.041188, 0.015395, 0.029396],
       [0.019139, 0.018357, 0.069164, 0.016344, 0.031006, 0.033433,
        0.038168, 0.032852, 0.025877, 0.013959, 0.013763, 0.02224 ],
       [0.012855, 0.012294, 0.016344, 0.111396, 0.015362, 0.021428,
        0.024784, 0.017683, 0.01566 , 0.015072, 0.01172 , 0.015285],
       [0.016971, 0.028692, 0.031006, 0.015362, 0.0541  , 0.027188,
        0.045185, 0.040054, 0.019921, 0.023303, 0.014767, 0.021559],
       [0.045822, 0.034565, 0.033433, 0.021428, 0.027188, 0.131522,
        0.080148, 0.04737 , 0.039233, 0.028484, 0.016177, 0.040736],
       [0.050579, 0.066176, 0.038168, 0.024784, 0.045185, 0.080148,
        0.332083, 0.08939 , 0.052683, 0.061946, 0.014247, 0.0556  ],
       [0.028011, 0.048799, 0.032852, 0.0

In [61]:
# Expected Returns Vector

expected_returns = np.array([0.228947982, 0.143959519, 0.193394501, 0.178632893, 0.185163027, 0.158488054, 0.213786214, 0.209991922, 0.188074578, 0.296524258, 0.165649812, 0.08]) # From excel

expected_returns

array([0.22894798, 0.14395952, 0.1933945 , 0.17863289, 0.18516303,
       0.15848805, 0.21378621, 0.20999192, 0.18807458, 0.29652426,
       0.16564981, 0.08      ])

In [62]:
# Beta Vector
betas = cov_matrix[-1] / cov_matrix[-1,-1]

betas

array([0.94449388, 1.0899518 , 0.82461995, 0.56674082, 0.79936967,
       1.51041898, 2.06154987, 1.28186874, 0.98806081, 0.89555061,
       0.48657768, 1.        ])

In [63]:
# Portfolio metric functions
def portfolio_xs_return(weights, expected_returns):
    bmk_return = expected_returns[-1]
    return weights @ expected_returns - bmk_return

def portfolio_beta(weights, betas):
    return weights @ betas

def portfolio_tracking_error(current_weights, cov_matrix):
    active_weights = current_weights.copy()
    active_weights[-1] -= 1
    return np.sqrt(active_weights @ cov_matrix @ active_weights)

def negative_information_ratio(current_weights, expected_returns, cov_matrix):
    port_er = portfolio_xs_return(current_weights, expected_returns)
    port_te = portfolio_tracking_error(current_weights, cov_matrix)
    return -port_er / port_te

In [64]:
# Display Weights Function
def display_weights(tickers, weights):
    df = pd.DataFrame()
    df['Ticker'] = tickers
    df['Weight'] = [round(weight, 2) for weight in weights]

    display(df)

In [65]:
# Baseline results

# Initial Weights
size = len(tickers)
initial_weights = np.array([0.0161, 0.0246, 0.0010, 0.0488, 0.0297, 0.0489, 0.0255, 0.0267, 0.0342, 0.0219, 0.0341, 0.6879])

# Baseline Results
port_return = portfolio_xs_return(initial_weights, expected_returns)
port_tracking_error = portfolio_tracking_error(initial_weights, cov_matrix)
port_ir = port_return / port_tracking_error

#Display
display_weights(tickers, initial_weights)
print(f"Portfolio Return: {round(port_return,2)} %")
print(f"Portfolio Tracking Error: {round(port_tracking_error,2)} %")
print(f"Portfolio Information Ratio: {round(port_ir,3)}")
print(f"Weights sum to {round(initial_weights.sum(),2)}")

,Ticker,Weight
0,AAPL,0.02
1,C,0.02
2,CMG,0.00
3,COKE,0.05
4,DRI,0.03
5,KLAC,0.05
6,MP,0.03
7,MTN,0.03
8,NFLX,0.03
9,GEF,0.02


Portfolio Return: 0.03 %
Portfolio Tracking Error: 0.03 %
Portfolio Information Ratio: 1.169
Weights sum to 1.0


In [66]:
# Optimization

# Constraints and bounds
constraints = [
    {'type': 'eq', 'fun': lambda x: np.sum(x) - 1},  # weights sum to 1
    {'type': 'eq', 'fun': lambda x: portfolio_beta(x, betas) - 1},  # beta of 1
    {'type': 'eq', 'fun': lambda x: portfolio_tracking_error(x, cov_matrix) - .05},  # tracking error of .05
  ]

# Result
result = minimize(negative_information_ratio, initial_weights, 
                  args=(expected_returns, cov_matrix),
                  method='SLSQP', constraints=constraints)

optimal_weights = result.x

optimal_weights

array([0.11878134, 0.02865374, 0.06515954, 0.02000632, 0.03694984,
       0.06637312, 0.02940106, 0.04317779, 0.03952884, 0.10100506,
       0.06761082, 0.38335253])

In [67]:
# Optimal Results

opt_port_tracking_error = portfolio_tracking_error(optimal_weights,cov_matrix)
opt_port_return = portfolio_xs_return(optimal_weights, expected_returns)
opt_port_beta = portfolio_beta(optimal_weights, betas)
opt_port_information_ratio = opt_port_return / opt_port_tracking_error

# Display
display_weights(tickers,optimal_weights)
print(f"Optimized Portfolio Beta: {round(opt_port_beta,2)}")
print(f"Optimized Portfolio Return: {round(opt_port_return,2) } %")
print(f"Optimized Portfolio Tracking Error: {round(opt_port_tracking_error,2)}")
print(f"Optimized Portfolio Information Ratio: {round(opt_port_information_ratio,2)}")
print(f"Weights sum to: {round(optimal_weights.sum(), 2)}")

,Ticker,Weight
0,AAPL,0.12
1,C,0.03
2,CMG,0.07
3,COKE,0.02
4,DRI,0.04
5,KLAC,0.07
6,MP,0.03
7,MTN,0.04
8,NFLX,0.04
9,GEF,0.10


Optimized Portfolio Beta: 1.0
Optimized Portfolio Return: 0.08 %
Optimized Portfolio Tracking Error: 0.05
Optimized Portfolio Information Ratio: 1.59
Weights sum to: 1.0
